In [69]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import struct
import os
from tqdm import tqdm
from tqdm import trange
from tensorflow.python.client import device_lib

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Parameters
learning_rate = 0.01
train_epochs = 40
batch_size = 5000

# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128 # 2nd layer num features
num_hidden_3 = 64 # 3rd layer num features
num_input = 784 # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b3': tf.Variable(tf.random_normal([num_input])),
}

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [70]:
def load_mnist_train(path, kind='train'): 
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath,dtype=np.uint8)
    with open(images_path, 'rb') as imgpath: 
        magic, num, rows, cols = struct.unpack('>IIII',imgpath.read(16)) 
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels), 784)
    
    return images, labels

In [71]:
def load_mnist_test(path, kind='t10k'): 
    labels_path = os.path.join(path,'%s-labels.idx1-ubyte'% kind) 
    images_path = os.path.join(path,'%s-images.idx3-ubyte'% kind) 
    with open(labels_path, 'rb') as lbpath: 
        magic, n = struct.unpack('>II',lbpath.read(8)) 
        labels = np.fromfile(lbpath,dtype=np.uint8) 
    with open(images_path, 'rb') as imgpath: 
        magic, num, rows, cols = struct.unpack('>IIII',imgpath.read(16)) 
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels), 784)
    
    return images, labels

In [72]:
path = os.getcwd()
train_images, train_labels = load_mnist_train(path)
test_images, test_labels = load_mnist_test(path)
print(len(train_images))

60000


In [73]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']), name = 'feature')
    return layer_3


In [74]:
# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    # Decoder Hidden layer with sigmoid activation #3
    layer_3 = tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                biases['decoder_b3'])
    return layer_3

In [75]:
def check_available_gpus():
    local_devices = device_lib.list_local_devices()
    gpu_names = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
    gpu_num = len(gpu_names)
    print('{0} GPUs are detected : {1}'.format(gpu_num, gpu_names))
    return gpu_num

In [76]:
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X
#Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(y_true- y_pred,2),name = 'loss')
#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
saver = tf.train.Saver()
check_available_gpus()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

1 GPUs are detected : ['/device:GPU:0']


In [83]:
if not os.path.exists('tmpnn/'):
    os.mkdir('tmpnn/')
with tf.Session() as sess:
    if os.path.exists('tmpnn/checkpoint'):
        saver.restore(sess, 'tmpnn/model.ckpt')
    else:
        sess.run(tf.global_variables_initializer())
    total_batch  = int(len(train_images)/batch_size)
    min_lose = float('inf')
    for e in range(train_epochs):    
        for b in tqdm(range(total_batch)):
            batch = train_images[b*batch_size:(b+1)*batch_size]
            imgs = batch.reshape((-1,28,28,1))
            batch_cost, _=sess.run([loss, optimizer],feed_dict={X:batch})
            msg = "Global Step={:d}, Local batch={:d}, lose={:.4f}"
            if(batch_cost<min_lose):
                min_lose = batch_cost
                saver.save(sess, 'tmpnn/model.ckpt')
            print(msg.format(e,b,batch_cost))
    

INFO:tensorflow:Restoring parameters from tmpnn/model.ckpt


 50%|█████     | 6/12 [00:00<00:03,  1.88it/s]

Global Step=0, Local batch=0, lose=1695.0353
Global Step=0, Local batch=1, lose=1711.2780
Global Step=0, Local batch=2, lose=1733.2722
Global Step=0, Local batch=3, lose=1733.5719
Global Step=0, Local batch=4, lose=1738.7422
Global Step=0, Local batch=5, lose=1724.7974
Global Step=0, Local batch=6, lose=1722.5214
Global Step=0, Local batch=7, lose=1743.2502
Global Step=0, Local batch=8, lose=1732.9133
Global Step=0, Local batch=9, lose=1784.8668


 50%|█████     | 6/12 [00:00<00:00, 53.32it/s]

Global Step=0, Local batch=10, lose=1704.7642
Global Step=0, Local batch=11, lose=1712.7388
Global Step=1, Local batch=0, lose=1700.7657
Global Step=1, Local batch=1, lose=1706.4130
Global Step=1, Local batch=2, lose=1733.9415
Global Step=1, Local batch=3, lose=1726.8740
Global Step=1, Local batch=4, lose=1727.2826
Global Step=1, Local batch=5, lose=1721.6180
Global Step=1, Local batch=6, lose=1713.7242
Global Step=1, Local batch=7, lose=1733.5671
Global Step=1, Local batch=8, lose=1734.9775
Global Step=1, Local batch=9, lose=1779.0571


...

Global Step=38, Local batch=11, lose=1575.6924


 58%|█████▊    | 7/12 [00:00<00:01,  3.00it/s]

Global Step=39, Local batch=0, lose=1573.1053
Global Step=39, Local batch=1, lose=1585.6573
Global Step=39, Local batch=2, lose=1597.9255
Global Step=39, Local batch=3, lose=1593.3154
Global Step=39, Local batch=4, lose=1590.5841
Global Step=39, Local batch=5, lose=1588.5830
Global Step=39, Local batch=6, lose=1590.1224
Global Step=39, Local batch=7, lose=1604.4999
Global Step=39, Local batch=8, lose=1591.8671
Global Step=39, Local batch=9, lose=1640.9312
Global Step=39, Local batch=10, lose=1576.8514


100%|██████████| 12/12 [00:01<00:00,  3.78it/s]

Global Step=39, Local batch=11, lose=1569.6067


In [84]:
#Test Processing:
output = []
with tf.Session() as sess:
    if os.path.exists('tmpnn/checkpoint'):        
        saver.restore(sess, 'tmpnn/model.ckpt')
        total_batch  = int(len(test_images)/batch_size)
        msg = "Test batch={:d}, lose={:.4f}"
        for b in tqdm(range(total_batch)):
            batch = test_images[b*batch_size:(b+1)*batch_size]
            imgs = batch.reshape((-1,28,28,1))
            cost,out = sess.run([loss,y_pred], feed_dict={input_x:batch})
            output.append(out)
            print(msg.format(b,cost))

    else :
        print("Please run training part first")
    fig1 = plt.figure(figsize=(8,8))
    fig1.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.5, wspace=0.05)
    for i in range(30):
        image = np.reshape(test_images[i],[28,28])
        ax=fig1.add_subplot(6,5,i+1,xticks=[],yticks=[]) 
        ax.imshow(image,cmap=plt.cm.binary,interpolation='nearest') 
        ax.text(0,7,str(test_labels[i]))
    plt.show()
    print("---------------------------------------------------")
    fig = plt.figure(figsize=(8,8))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.5, wspace=0.05)
    for i in range(30):
        image = np.reshape(output[0][i],[28,28])
        ax=fig.add_subplot(6,5,i+1,xticks=[],yticks=[]) 
        ax.imshow(image,cmap=plt.cm.binary,interpolation='nearest') 
        ax.text(0,7,str(test_labels[i]))
    plt.show()

INFO:tensorflow:Restoring parameters from tmpnn/model.ckpt


  0%|          | 0/2 [00:00<?, ?it/s]


NameError: name 'input_x' is not defined